# 含并行连结的网络（GoogLeNet）

在2014年的ImageNet图像识别挑战赛中，一个名叫GoogLeNet的网络结构大放异彩 [1]。它虽然在名字上向LeNet致敬，但在网络结构上已经很难看到LeNet的影子。GoogLeNet吸收了NiN中网络串联网络的思想，并在此基础上做了很大改进。在随后的几年里，研究人员对GoogLeNet进行了数次改进，本节将介绍这个模型系列的第一个版本。


## Inception 块

GoogLeNet中的基础卷积块叫作Inception块，得名于同名电影《盗梦空间》（Inception）。与上一节介绍的NiN块相比，这个基础块在结构上更加复杂，如图5.8所示。

![Inception块的结构](../img/inception.svg)

由图5.8可以看出，Inception块里有4条并行的线路。前3条线路使用窗口大小分别是$1\times 1$、$3\times 3$和$5\times 5$的卷积层来抽取不同空间尺寸下的信息，其中中间2个线路会对输入先做$1\times 1$卷积来减少输入通道数，以降低模型复杂度。第四条线路则使用$3\times 3$最大池化层，后接$1\times 1$卷积层来改变通道数。4条线路都使用了合适的填充来使输入与输出的高和宽一致。最后我们将每条线路的输出在通道维上连结，并输入接下来的层中去。

Inception块中可以自定义的超参数是每个层的输出通道数，我们以此来控制模型复杂度。

In [1]:
import d2lzh as d2l
from mxnet import gluon, init, nd
from mxnet.gluon import nn

class Inception(nn.Block):
    # c1 - c4为每条线路里的层的输出通道数
    def __init__(self, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # 线路1，单1 x 1卷积层
        self.p1_1 = nn.Conv2D(c1, kernel_size=1, activation='relu')
        # 线路2，1 x 1卷积层后接3 x 3卷积层
        self.p2_1 = nn.Conv2D(c2[0], kernel_size=1, activation='relu')
        self.p2_2 = nn.Conv2D(c2[1], kernel_size=3, padding=1,
                              activation='relu')
        # 线路3，1 x 1卷积层后接5 x 5卷积层
        self.p3_1 = nn.Conv2D(c3[0], kernel_size=1, activation='relu')
        self.p3_2 = nn.Conv2D(c3[1], kernel_size=5, padding=2,
                              activation='relu')
        # 线路4，3 x 3最大池化层后接1 x 1卷积层
        self.p4_1 = nn.MaxPool2D(pool_size=3, strides=1, padding=1)
        self.p4_2 = nn.Conv2D(c4, kernel_size=1, activation='relu')

    def forward(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        return nd.concat(p1, p2, p3, p4, dim=1)  # 在通道维上连结输出

## GoogLeNet模型

GoogLeNet跟VGG一样，在主体卷积部分中使用5个模块（block），每个模块之间使用步幅为2的$3\times 3$最大池化层来减小输出高宽。第一模块使用一个64通道的$7\times 7$卷积层。

In [2]:
b1 = nn.Sequential()
b1.add(nn.Conv2D(64, kernel_size=7, strides=2, padding=3, activation='relu'),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

第二模块使用2个卷积层：首先是64通道的$1\times 1$卷积层，然后是将通道增大3倍的$3\times 3$卷积层。它对应Inception块中的第二条线路。

In [3]:
b2 = nn.Sequential()
b2.add(nn.Conv2D(64, kernel_size=1, activation='relu'),
       nn.Conv2D(192, kernel_size=3, padding=1, activation='relu'),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

第三模块串联2个完整的Inception块。第一个Inception块的输出通道数为$64+128+32+32=256$，其中4条线路的输出通道数比例为$64:128:32:32=2:4:1:1$。其中第二、第三条线路先分别将输入通道数减小至$96/192=1/2$和$16/192=1/12$后，再接上第二层卷积层。第二个Inception块输出通道数增至$128+192+96+64=480$，每条线路的输出通道数之比为$128:192:96:64 = 4:6:3:2$。其中第二、第三条线路先分别将输入通道数减小至$128/256=1/2$和$32/256=1/8$。

In [4]:
b3 = nn.Sequential()
b3.add(Inception(64, (96, 128), (16, 32), 32),
       Inception(128, (128, 192), (32, 96), 64),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

第四模块更加复杂。它串联了5个Inception块，其输出通道数分别是$192+208+48+64=512$、$160+224+64+64=512$、$128+256+64+64=512$、$112+288+64+64=528$和$256+320+128+128=832$。这些线路的通道数分配和第三模块中的类似，首先含$3\times 3$卷积层的第二条线路输出最多通道，其次是仅含$1\times 1$卷积层的第一条线路，之后是含$5\times 5$卷积层的第三条线路和含$3\times 3$最大池化层的第四条线路。其中第二、第三条线路都会先按比例减小通道数。这些比例在各个Inception块中都略有不同。

In [5]:
b4 = nn.Sequential()
b4.add(Inception(192, (96, 208), (16, 48), 64),
       Inception(160, (112, 224), (24, 64), 64),
       Inception(128, (128, 256), (24, 64), 64),
       Inception(112, (144, 288), (32, 64), 64),
       Inception(256, (160, 320), (32, 128), 128),
       nn.MaxPool2D(pool_size=3, strides=2, padding=1))

第五模块有输出通道数为$256+320+128+128=832$和$384+384+128+128=1024$的两个Inception块。其中每条线路的通道数的分配思路和第三、第四模块中的一致，只是在具体数值上有所不同。需要注意的是，第五模块的后面紧跟输出层，该模块同NiN一样使用全局平均池化层来将每个通道的高和宽变成1。最后我们将输出变成二维数组后接上一个输出个数为标签类别数的全连接层。

In [6]:
b5 = nn.Sequential()
b5.add(Inception(256, (160, 320), (32, 128), 128),
       Inception(384, (192, 384), (48, 128), 128),
       nn.GlobalAvgPool2D())

net = nn.Sequential()
net.add(b1, b2, b3, b4, b5, nn.Dense(10))

GoogLeNet模型的计算复杂，而且不如VGG那样便于修改通道数。本节里我们将输入的高和宽从224降到96来简化计算。下面演示各个模块之间的输出的形状变化。

In [7]:
X = nd.random.uniform(shape=(1, 1, 96, 96))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

sequential0 output shape:	 (1, 64, 24, 24)
sequential1 output shape:	 (1, 192, 12, 12)
sequential2 output shape:	 (1, 480, 6, 6)
sequential3 output shape:	 (1, 832, 3, 3)
sequential4 output shape:	 (1, 1024, 1, 1)
dense0 output shape:	 (1, 10)


## 获取数据和训练模型

我们使用高和宽均为96像素的图像来训练GoogLeNet模型。训练使用的图像依然来自Fashion-MNIST数据集。

In [8]:
lr, num_epochs, batch_size, ctx = 0.1, 5, 128, d2l.try_gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx,
              num_epochs)

training on gpu(0)
epoch 1, loss 1.6641, train acc 0.365, test acc 0.709, time 25.4 sec
epoch 2, loss 0.6860, train acc 0.745, test acc 0.809, time 22.2 sec
epoch 3, loss 0.5150, train acc 0.810, test acc 0.851, time 22.3 sec
epoch 4, loss 0.3856, train acc 0.855, test acc 0.865, time 22.4 sec
epoch 5, loss 0.3347, train acc 0.874, test acc 0.882, time 22.2 sec


## 小结

* Inception块相当于一个有4条线路的子网络。它通过不同窗口形状的卷积层和最大池化层来并行抽取信息，并使用$1\times 1$卷积层减少通道数从而降低模型复杂度。
* GoogLeNet将多个设计精细的Inception块和其他层串联起来。其中Inception块的通道数分配之比是在ImageNet数据集上通过大量的实验得来的。
* GoogLeNet和它的后继者们一度是ImageNet上最高效的模型之一：在类似的测试精度下，它们的计算复杂度往往更低。

## 练习

* GoogLeNet有数个后续版本。尝试实现并运行它们，然后观察实验结果。这些后续版本包括加入批量归一化层（下一节将介绍）[2]、对Inception块做调整 [3] 和加入残差连接（[“残差网络（ResNet）”](resnet.ipynb)一节将介绍）[4]。

* 对比AlexNet、VGG和NiN、GoogLeNet的模型参数尺寸。为什么后两个网络可以显著减小模型参数尺寸？




## 参考文献

[1] Szegedy, C., Liu, W., Jia, Y., Sermanet, P., Reed, S., & Anguelov, D. & Rabinovich, A.(2015). Going deeper with convolutions. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 1-9).

[2] Ioffe, S., & Szegedy, C. (2015). Batch normalization: Accelerating deep network training by reducing internal covariate shift. arXiv preprint arXiv:1502.03167.

[3] Szegedy, C., Vanhoucke, V., Ioffe, S., Shlens, J., & Wojna, Z. (2016). Rethinking the inception architecture for computer vision. In Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition (pp. 2818-2826).

[4] Szegedy, C., Ioffe, S., Vanhoucke, V., & Alemi, A. A. (2017, February). Inception-v4, inception-resnet and the impact of residual connections on learning. In Proceedings of the AAAI Conference on Artificial Intelligence (Vol. 4, p. 12).

## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/1662)

![](../img/qr_googlenet.svg)

## 个人总结与练习
### 个人总结
**inception-v1**
论文：https://www.dropbox.com/s/hdc8faj523wzb8y/going%20deeper%20with%20convlotions.pdf?dl=0
- inception-v1中收了NIN的启发利用bottle layer来近似 稀疏计算。传统的CNN通过convolution操作来实现局部稀疏计算，但是它没能实现在layer层面的完全稀疏化；ConvNet通过一种 `sparse connection table` 在feature层面上选择部分的feature作为下层的计算，计算效率低。而inception-v1就设想通过三种尺度的filter实现部分的dense计算合并来模拟全局的sparse计算。
- inception block在三个方面提高计算效率：稀疏矩阵的计算、多种尺度特征的提取、Hebbin理论下提前将强相关的feature group在一起计算。能够提取出更加丰富的特征，也实现了计算高效增强收敛速度。
- 使用NIN中的 `1*1` layer来降低channel的维度，将整个网络中的参数量减小。注意在dimension reduction的时机，在feature提取之前，如果在提取之后对embedding进行压缩不利于后层的处理。
- GoogLeNet整体结构上前三层使用传统CNN设计后层使用inception block设计，随着层次的加深 结构中的sparsity connection也在降低这就需要增加大尺寸filter的数量。
- 训练上解决gradient vanishing通过在底层中分别增加两个分类器，将底层的loss加入到整体网络的loss中，boost网络gradient，实际上的效果提升不是十分的明显。

**batch norm**
论文：https://www.dropbox.com/s/61k505fi7ji45k5/Batch-Normalization-Accelerating-Deep-Network-Training-by-Reducing-Internal-Covariate-Shift.pdf?dl=0
- batch norm在数据的分布角度入手解决深度网络难训练的问题，由于层层耦合的非线性变化使得层层之间的数据分布大，难以训练，非常容易出现gradiet vanishing与explosion的问题。
- BN正对activation前的数据进行normalization，让数据同分布；但是在transformation时候得保证当前层identity保留，如果只是进行简单的normalization在BP算法学习weight的时候并没有去考虑到normalization发生，其中的weight会出现gradient为0的请款。BN提出的结局方案是在简单normalization的基础上增加 `learned shift var and rescale var` 相同的activation对用的feature共享学习一组shift var与scale var。以此来保证normalization被学习系统考虑入内。
- 加入了BN算法的网络训练速度大大加快，能够用比较大的学习率加速学习，它也具有Regularization的功能；对于那些由于数据分布稳定向影响的saturating nonlinearty 在BN下也可以使用。
- 注意在inference的时刻结合 learned shift and scale var的同时也会对整个的inference sample 做normalization。

**incetion-v3**
论文：https://www.dropbox.com/s/xq6zg5zgcrgj4jj/Szegedy_Rethinking_the_Inception_CVPR_2016_paper.pdf?dl=0
- 思考通过factorizing convolution的方法来降低inception block的参数量，构建了新的inception block；使用新block 可以构建出更深的网络。
- 本质上是渐进式的改良，在feature grid size的降维设计了行的block来进行，而不是直接使用pooling的方式，避免了representation bottleneck的出现，进一步提升网络表现力。
- 使用LRS 在样本标签上的分布上引入先验知识来降低overfiting。
- 优化auxiliary classifer在中加入BN与dropout。

**inception—v4**
论文：https://www.dropbox.com/s/jlcz3p2u394alvv/Inception-v4%2C%20inception-resnet%20and%20the%20impact%20of%20residual%20connections%20on%20learning.%20In%20Proceedings%20of%20the%20AAAI%20Conference%20on%20Artificial%20Intelligence%20.pdf?dl=0
- 利用了residual设计res-inception net。
- 在原来的inception-v3基础上进一步改性，在block上设计更优。与res版本控制对比实验验证residual的功效。
- inception-v4与res版本在模型表现力上是相似的，而res版本的训练速度更快。验证了res确实对网络训练提速有效，由于没有出现degradation。inception block的设计更「精细」，对比vgg Block它可以学到更加准确分离的feature，通过inception block来构建深度网络时不会出现degradation。实验中的res的深度控制在与inception相同，没有验证计生res的深度是否效果会更佳，或者作者实验了但是没有在论文中提及。
-  residual对深度网络设计时必要这样的观点在inception-v4中没有得到支持，需要进一步研究深度网络中degradation出现的原因，及其本质解决方法。

### 练习
- 实现网络的设计
- NiN、inception的结构在本质上在conv时减少了feature map的dimension使用了合适的压缩方法；这样使得cov的filter数量较少，在大量的layer叠加的时候极大减少了参数量，对比VGG inception-v1减少有19倍之多。